# Agenda:

In this final session on the topic of data analysis in `Python`, we will an example of how we can work with a dataset and perform some analysis steps to answer questions about the data in hand.

You can imagine this as a typical task of data scientist. Reading and manipulating data and reaching some useful insights.

Following the previous sessions, we will continue using the `pandas` library, and we will introduce three new functions:

1. `groupby`: for calculating summary statistics (min, max, etc) `group-wise`, rather than on the whole data.
2. `merge`: for merging (joining) two dataframes into one based on a common key.
3. `pivot`: for calculating [pivot table](https://en.wikipedia.org/wiki/Pivot_table).

# Import libraries

Uncomment the following cell to install the libraries:

In [ ]:
# ! pip install pandas numpy plotly

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# Reading data in pands:

There are plenty of functions for loading _tabular data_ in pandas, including:

1. `read_csv`
2. `read_excel`
3. `read_html`
4. `read_json`
5. `read_sql`

Depending on the file type of your data, you use the appropriate function.

# Load data

The data we are working with in this notebook is hosted on Kaggle: [Google Play Store Apps](https://www.kaggle.com/datasets/lava18/google-play-store-apps).

It consists of two data files:

1. apps information.
2. apps reviews.

In [ ]:
google_play_apps_df = pd.read_csv(
    "https://raw.githubusercontent.com/Reslan-Tinawi/python-data-analysis-workshop/main/4-case-study/data/google-play-store-data-cleaned.csv"
)

In [ ]:
google_play_apps_reviews_df = pd.read_csv(
    "https://raw.githubusercontent.com/Reslan-Tinawi/python-data-analysis-workshop/main/4-case-study/data/google-play-store-user-reviews-cleaned.csv"
)

In [ ]:
google_play_apps_df.head()

In [ ]:
google_play_apps_reviews_df.head()

In [ ]:
google_play_apps_df.shape

In [ ]:
google_play_apps_reviews_df.shape

# Data description:

1. `App`: Application name
2. `Category`: Category the app belongs to
3. `Rating`: Overall user rating of the app (as when scraped)
4. `Reviews`: Number of user reviews for the app (as when scraped)
5. `Size`: Size of the app (as when scraped)
6. `Installs`: Number of user downloads/installs for the app (as when scraped)
7. `Type`: Paid or Free
8. `Price`: Price of the app (as when scraped)
9. `Content Rating`: Age group the app is targeted at - Children / Mature 21+ / Adult
10. `Genres`: An app can belong to multiple genres (apart from its main category). For eg, a musical family game will belong to
11. `Last Updated`: Date when the app was last updated on Play Store (as when scraped)
12. `Current Ver`: Current version of the app available on Play Store (as when scraped)
13. `Android Ver`: Min required Android version (as when scraped)

1. `Translated_Review`: User review (Preprocessed and translated to English)
2. `Sentiment`: Positive/Negative/Neutral (Preprocessed)
3. `Sentiment_Polarity`: Sentiment polarity score
4. `Sentiment_Subjectivity`: Sentiment subjectivity score

# Simple questions

## How many different categories are there?

In [ ]:
google_play_apps_df["Category"].nunique()

# What are the unique categories?

In [ ]:
google_play_apps_df["Category"].unique()

# What is the highest/lowest rating app(s)?

In [ ]:
min_rating_value = google_play_apps_df["Rating"].min()

In [ ]:
print(min_rating_value)

In [ ]:
google_play_apps_df["Rating"] == min_rating_value

In [ ]:
google_play_apps_df.loc[google_play_apps_df["Rating"] == min_rating_value]

In [ ]:
max_rating_value = google_play_apps_df["Rating"].max()

In [ ]:
print(max_rating_value)

In [ ]:
google_play_apps_df.loc[google_play_apps_df["Rating"] == max_rating_value]

# Android market breakdown:

The number of active apps per category:

In [ ]:
google_play_apps_df["Category"].value_counts(normalize=True)

In simple words, the `groupby` function performs an aggregation functions group-wise.

In our case, when we _group by_ `Category` is as saying:

**Perform this aggregation function(s) for each _distinct_ category independently**

The simplest case is calculating the total number of rows for each category. In this case our aggregation function is `count` and we are grouping by `Category`

In [ ]:
category_stats_df = google_play_apps_df.groupby(
    by="Category", as_index=False
).aggregate(number_of_apps=pd.NamedAgg(column="Rating", aggfunc="count"))

In [ ]:
category_stats_df.head()

We can sort dataframe rows by column values. Here, we sort by the `number_of_apps` colum in descending order (highest value comes first):

In [ ]:
category_stats_df = category_stats_df.sort_values(by="number_of_apps", ascending=False)

In [ ]:
category_stats_df.head()

In [ ]:
fig = px.bar(data_frame=category_stats_df, x="Category", y="number_of_apps")

fig.show()

# How apps are rated on general?

In [ ]:
google_play_apps_df["Rating"].mean()

In [ ]:
google_play_apps_df["Rating"].max()

In [ ]:
fig = px.histogram(data_frame=google_play_apps_df, x="Rating")

fig.update_traces(xbins=dict(start=0.0, end=5.0, size=0.5))  # bins used for histogram

fig.show()

# How apps rating change by app category?

In [ ]:
google_play_apps_df[["Category", "Rating"]]

Let's calculate min, max and average rating for each category to understand the how these measurements change by app category.

Here, we need to perform `groupby` as we are calculating an aggregate function for each group separately.

In [ ]:
category_rating_stats_df = google_play_apps_df.groupby(
    by="Category", as_index=False
).aggregate(
    category_min_rating=pd.NamedAgg(  # min aggregate function
        column="Rating", aggfunc="min"
    ),
    category_max_rating=pd.NamedAgg(  # max aggregate function
        column="Rating", aggfunc="max"
    ),
    category_average_rating=pd.NamedAgg(  # mean aggregate function
        column="Rating", aggfunc="mean"
    ),
)

Notice how aggregations are performed:
- we define the column over which the aggrgation function is performed: `column`
- and we then specify the aggregate function we wish to calculate: `aggfunc`

This code:

`pd.NamedAgg(column="Rating", aggfunc="mean")`

means:

**Calculate the mean of `Rating` column for each group.**

In [ ]:
category_rating_stats_df.head(10)

Now, we have for each _category_ (`ART_AND_DESIGN`, `AUTO_AND_VEHICLES`, `BEAUTY`, etc ...) three measurements:

1. `category_min_rating`: minimum rating value for the category
2. `category_max_rating`: maximum rating value for the category
3. `category_average_rating`: average rating for the category

The following image better explains how the `groupby` function works:

<div>
    <img src="img/group-by-example.png" alt='numpy-array-vs-pandas-series' width="1000"/>
</div>

Let's visualize the `Rating` histogram for only few categories to compare them:

In [ ]:
focus_categories = [
    "BOOKS_AND_REFERENCE",
    "BUSINESS",
    "COMMUNICATION",
    "EDUCATION",
    "ENTERTAINMENT",
]

Here, we use the `.isin` series function.

For each value in the `Category` column, if it exists in the `focus_categories` list, the function returns `True` and `False` otherwise.

This results in a boolean mask which we can use to filter the data.

In [ ]:
google_play_apps_df["Category"].isin(focus_categories)

In [ ]:
focus_app_data = data_frame = google_play_apps_df.loc[
    google_play_apps_df["Category"].isin(focus_categories)
]

In [ ]:
focus_app_data

In [ ]:
fig = px.histogram(
    data_frame=focus_app_data,
    x="Rating",
    color="Category",
    facet_row="Category",
    height=1000,
)

fig.show()

# What is the relation between app size and rating?

Is there a relation between app size and its rating?

For example, can we infer that bigger apps have higher rating?

In [ ]:
google_play_apps_df[["Rating", "Size"]]

Scatter plots are useful for studying the binary relation between two _numerical_ variables.

In [ ]:
fig = px.scatter(data_frame=google_play_apps_df, x="Size", y="Rating")

fig.show()

Most top rated apps are optimally sized between ~2MB to ~40MB - neither too light nor too heavy.

# App reviews

## Review sentiment

In [ ]:
google_play_apps_reviews_df.head()

In [ ]:
google_play_apps_reviews_df["Sentiment"].value_counts()

The `Sentiment` has three values:

1. `Positive`: the app review is positive.
2. `Neutral`: the app review is neutral.
3. `Negative`: the app review is negative.



## Merging

The `App` column is common between the two dataframes. We can _merge_ the two dataframes using this columns, to get one dataframe.

In [ ]:
google_play_apps_df.head(2)

In [ ]:
google_play_apps_reviews_df.head(2)

In the `merge` function, we specify the `left` and `right` dataframes:

In [ ]:
apps_and_reviews_df = pd.merge(
    left=google_play_apps_df, right=google_play_apps_reviews_df
)

In [ ]:
apps_and_reviews_df.head()

We can see that columns from `apps_and_reviews_df` have been added to the `google_play_apps_df`.

## App sentiment values

After merging the data, now we can inspect the `Sentiment` values for each app:

In [ ]:
apps_and_reviews_df[["App", "Sentiment"]]

Right now, the data is in _long format_, for each app we have several different sentiment values.

An easier way to look at the data is to transform it into _wide format_, where the different Sentiment values become columns with their values aggregated by app.

**NOTE**: the following code cell will only work if your `pandas` version is `1.4.2` or higher. At the time of developing this notebook, `pandas` version is `1.3.5`, therfore, the following code cell will raise an excpetion.

In [ ]:
apps_sentiment_df = (
    apps_and_reviews_df.groupby(by="App", as_index=False)["Sentiment"]
    .value_counts()
    .pivot(index="App", columns="Sentiment", values="count")
    .sort_values(by=["Positive", "Neutral", "Negative"], ascending=False)
    .fillna(0)
)

In [ ]:
apps_sentiment_df

The following image explains how the `pivot` function works:

<div>
    <img src="img/pivot-example.png" alt='numpy-array-vs-pandas-series' width="1000"/>
</div>